!st model using one feature : funding_employees_ratio with a different preprocessing

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import pandas as pd
import numpy as np
import json
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

# Import new data + treat json format

In [32]:
def load_json_field(bad_json):
    regex = r"\w\'\w"
    subst = ""
    bad_json = re.sub(regex, subst, bad_json)
    bad_json = bad_json.replace("d' Arrouzat", "darrouzat")
    good_json = bad_json.replace("\'", "\"").replace("None", "null").replace("True", "true").replace("False", "false")
    
    return json.loads(good_json)

In [33]:
data = pd.read_csv("bpideep/rawdata/data2020-12-03.csv")
data["team"] = data["team"].apply(load_json_field)
data["industries"] = data["industries"].apply(load_json_field)
data["investors"] = data["investors"].apply(load_json_field)

In [34]:
data.shape

(1513, 94)

## check duplicates id and name -> remove duplicates according to data

+ 3 duplicated names but with different id : 

1/ Lalilo : 926521 (http://www.lalilo.com/) vs. 1787891 (http://lalilo.fr) -> same launch date, french website no longer exists + observation  almost filled by NAN + same obs as the .com -> drop the french Lalilo (1787891)

2/ Pixyl : 892048 vs 1893232 (different websites mentioned but same website page when launched) -> kept  892048 because more info + the Dealroom profile was verified by Dealroom team on Sept, 1st 2020 vs. pending verification

3/ NANOZ : 1836121 vs 1660543 -> kept both as different companies but the second one is German 

In [35]:
idcheck = pd.DataFrame(data.id.value_counts())
print(idcheck[idcheck.id != 1])

namecheck = pd.DataFrame(data.name.value_counts())
print(namecheck[namecheck.name != 1])

Empty DataFrame
Columns: [id]
Index: []
        name
NANOZ      2
Lalilo     2
Pixyl      2


In [36]:
data[data.name == "Lalilo"]

,id,name,path,tagline,about,url,website_url,twitter_url,facebook_url,linkedin_url,...,employee_12_months_growth_delta,kpi_summary,team,investors,fundings,traffic,similarweb_chart,job_openings,deep_or_not,target
247,926521,Lalilo,lalilo,A literacy web-based app for teachers that use...,Empowering K-6 teachers with AI to personalize...,https://app.dealroom.co/companies/lalilo,http://www.lalilo.com/,https://twitter.com/laliloapp,NaN,https://www.linkedin.com/company/10939758,...,-1508.0,"{'id': 12776, 'last_update_date': '2020-09-30T...","{'items': [{'id': 1074541, 'name': 'Amine Mezz...","{'items': [{'id': 885471, 'name': 'Agoranov', ...","{'items': [{'id': 297571, 'year': 2019, 'month...","{'visitors': '894989', 'annual_growth': None, ...","[{'date': '2016-11-01', 'value': 0}, {'date': ...",3,deeptech,1.0
623,1787891,Lalilo,lalilo_1,NaN,NaN,https://app.dealroom.co/companies/lalilo_1,http://lalilo.fr,NaN,NaN,NaN,...,NaN,"{'id': 1798864, 'last_update_date': '2020-03-0...","{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'visitors': '361', 'annual_growth': -71.16613...","[{'date': '2020-03-01', 'value': 1252}, {'date...",0,deeptech,1.0


In [37]:
data[data.name == "Pixyl"]

,id,name,path,tagline,about,url,website_url,twitter_url,facebook_url,linkedin_url,...,employee_12_months_growth_delta,kpi_summary,team,investors,fundings,traffic,similarweb_chart,job_openings,deep_or_not,target
340,892048,Pixyl,pixyl,Neuroimaging software that automaticaly analyz...,NaN,https://app.dealroom.co/companies/pixyl,http://pixylmedical.com/,https://twitter.com/pixylmedical,NaN,https://www.linkedin.com/company/pixyl-medical/,...,NaN,"{'id': 6854, 'last_update_date': '2020-10-30T1...","{'items': [{'id': 1989998, 'name': 'Senan Doyl...","{'items': [{'id': 868737, 'name': 'Crédit Agri...","{'items': [{'id': 33006, 'year': 2017, 'month'...","{'visitors': '0', 'annual_growth': None, 'sour...","[{'date': '2016-11-01', 'value': 375}, {'date'...",0,deeptech,1.0
510,1893232,Pixyl,pixyl_1,An AI-based technological solution for quantif...,NaN,https://app.dealroom.co/companies/pixyl_1,http://pixyl.ai,https://twitter.com/pixylmedical,NaN,https://www.linkedin.com/company/pixyl-medical,...,NaN,"{'id': 1858862, 'last_update_date': '2020-11-1...","{'items': [{'id': 1989998, 'name': 'Senan Doyl...","{'items': [{'id': 868735, 'name': 'IT-Translat...","{'items': [{'id': 393095, 'year': 2020, 'month...","{'visitors': '991', 'annual_growth': 3.3368091...","[{'date': '2020-08-01', 'value': 959}, {'date'...",0,deeptech,1.0


In [38]:
data[data.name == "NANOZ"]

,id,name,path,tagline,about,url,website_url,twitter_url,facebook_url,linkedin_url,...,employee_12_months_growth_delta,kpi_summary,team,investors,fundings,traffic,similarweb_chart,job_openings,deep_or_not,target
518,1836121,NANOZ,nanoz_1,NaN,NaN,https://app.dealroom.co/companies/nanoz_1,http://nanoz-group.eu/,https://twitter.com/nanozgroup,NaN,https://www.linkedin.com/company/nanoz,...,NaN,"{'id': 1827681, 'last_update_date': '2020-05-1...","{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'visitors': '206', 'annual_growth': -4.629629...","[{'date': '2020-06-01', 'value': 216}, {'date'...",0,deeptech,1.0
631,1660543,NANOZ,nanoz,Nanozentrum Kassel in Kaufungen – Fahrzeugaufb...,NaN,https://app.dealroom.co/companies/nanoz,http://nanozentrum-kassel.de,NaN,NaN,https://www.linkedin.com/company/nanozentrum-k...,...,NaN,"{'id': 1725789, 'last_update_date': '2019-05-2...","{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'visitors': '0', 'annual_growth': None, 'sour...","[{'date': '2018-12-01', 'value': 104}, {'date'...",0,deeptech,1.0


In [39]:
data.drop(data[(data.id == 1787891) | (data.id == 1893232)].index, inplace = True)

In [40]:
data.shape

(1511, 94)

## selecting columns

In [41]:
data[data.corporate_industries == "[]"]["deep_or_not"].value_counts()

non_deeptech       683
deeptech           636
almost_deeptech    177
Name: deep_or_not, dtype: int64

In [42]:
kept_columns = ['id', 'name', 'path', 'tagline', 'about', 'employees', 'employees_latest',
       'industries','technologies', 'growth_stage','tags', 'launch_year', 'launch_month', 'has_strong_founder',
       'has_super_founder', 'total_funding', 'total_funding_source',
       'last_funding', 'last_funding_source', 'company_status', 
       'team', 'investors', 'fundings', 'deep_or_not', 'target']

In [43]:
selected_data = data[kept_columns]

# Features selection

## funding_employee_ratio

Feature preprocessing

In [44]:
selected_data.employees_latest.isna().sum()

138

### creating a new column diplaying the industry with a matching table that simplifies the classification

In [45]:
pd.set_option('display.max_rows', 140)
pd.DataFrame(data[data.employees_latest.isna()][["name", "id", "industries","sub_industries", 'launch_year', "employees"]])

,name,id,industries,sub_industries,launch_year,employees
15,APSI 3D,1478165,"[{'id': 100023, 'name': 'energy'}, {'id': 1001...",['clean energy'],2013.0,2-10
32,Amypore,1682182,"[{'id': 1254, 'name': 'health'}]",['biotechnology'],NaN,NaN
50,Avril,1249456,"[{'id': 1264, 'name': 'fintech'}]",[],1983.0,5001-10000
86,Ciloa SAS,969382,"[{'id': 1254, 'name': 'health'}]",['biotechnology'],1996.0,NaN
87,Cinequant,5741,"[{'id': 100147, 'name': 'enterprise software'}]",[],2011.0,1
89,CLINIFIT,1743459,"[{'id': 1254, 'name': 'health'}]",[],2013.0,NaN
98,Crosslux,1464696,"[{'id': 100023, 'name': 'energy'}]",[],2011.0,11-50
126,DocTrackr,79761,"[{'id': 1262, 'name': 'security'}, {'id': 1264...",['data protection'],2011.0,11-50
141,Enancio,1681973,"[{'id': 100147, 'name': 'enterprise software'}]",[],2017.0,2-10
172,Fi2Med,1748070,"[{'id': 1254, 'name': 'health'}]",[],2016.0,1


In [46]:
def industries(x):
    '''
    function that extracts info from 'industries' column through mapping
    data['industries_list'] = data['industries'].map(lambda x: industries(x))
    '''

    industries_list = []
    industries = x
    for u in range(len(industries)):
            industries_list.append(industries[u]['name'])
    return industries_list

In [47]:
selected_data["industry"] = selected_data["industries"].map(lambda x: industries(x))
selected_data["industry"]

<ipython-input-47-57ea1d2aa2d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data["industry"] = selected_data["industries"].map(lambda x: industries(x))


0                    [health]
1           [health, fintech]
2                    [energy]
3                    [health]
4                    [health]
                ...          
1508            [home living]
1509    [enterprise software]
1510    [enterprise software]
1511                 [energy]
1512     [fintech, marketing]
Name: industry, Length: 1511, dtype: object

In [48]:
ind_match = pd.read_excel("bpideep/data/matching_industry.xlsx")

complicated = list(ind_match["industry"])
simplified = list(ind_match["industry_simplified"])

zipbObj = zip(complicated, simplified)
ind_dict = dict(zipbObj)

selected_data.industry = selected_data.industry.astype(str)


selected_data['industry_simplified'] = selected_data.industry.map(ind_dict)

# for complicate, simple in ind_dict.items() : 
#     data.loc[selected_data.industry == complicate, "industry_simplified"] = simple

/Users/catherinechen/.pyenv/versions/3.8.6/envs/deep4deep2/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-48-ce00b13bf3fd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['industry_simplified'] = selected_data.industry.map(ind_dict)


### Years of existence

In [49]:
def age(x):
    return 2020 - x

In [50]:
selected_data["age"] = selected_data["launch_year"].map(lambda x: age(x))

<ipython-input-50-ee599f56c516>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data["age"] = selected_data["launch_year"].map(lambda x: age(x))


In [51]:
selected_data.age.isna().sum()

90

In [52]:
selected_data[selected_data.age.isna() & selected_data.industry_simplified.isna()]

,id,name,path,tagline,about,employees,employees_latest,industries,technologies,growth_stage,...,last_funding_source,company_status,team,investors,fundings,deep_or_not,target,industry,industry_simplified,age
409,1468106,SurfactGreen,surfactgreen,Developing and producing performing cationics ...,NaN,11-50,11.0,[],[],early growth,...,3100000,operational,"{'items': [], 'total': 0}","{'items': [{'id': 882486, 'name': 'Finovam', '...","{'items': [{'id': 341837, 'year': 2020, 'month...",deeptech,1.0,[],NaN,NaN
439,894908,Tryon,tryon,Designs biogas plants at a very small scale to...,NaN,NaN,NaN,[],[],mature,...,0,operational,"{'items': [], 'total': 0}","{'items': [{'id': 885471, 'name': 'Agoranov', ...","{'items': [{'id': 222543, 'year': 2016, 'month...",deeptech,1.0,[],NaN,NaN
455,906637,voxolab,voxolab,Voxolab,NaN,1,1.0,[],[],seed,...,0,operational,"{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'items': [], 'total': 0}",deeptech,1.0,[],NaN,NaN
477,198955,Yealth,yealth,Yealth,"Yealth, connectez votre santé, restez autonome...",2-10,3.0,[],[],seed,...,0,operational,"{'items': [{'id': 312623, 'name': 'Thibaut Cam...","{'items': [], 'total': 0}","{'items': [], 'total': 0}",deeptech,1.0,[],NaN,NaN
520,1924305,Uniris,uniris,Uniris - Be the only key to the next evolution...,NaN,NaN,NaN,[],[],NaN,...,0,operational,"{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'items': [], 'total': 0}",deeptech,1.0,[],NaN,NaN
568,1832473,CRYONOVE PHARMA,cryonove_pharma,CRYOBEAUTY - Le 1er dispositif médical Anti-Ta...,NaN,1,1.0,[],[],seed,...,0,operational,"{'items': [], 'total': 0}","{'items': [{'id': 863894, 'name': 'GO Capital'...","{'items': [], 'total': 0}",deeptech,1.0,[],NaN,NaN
631,1660543,NANOZ,nanoz,Nanozentrum Kassel in Kaufungen – Fahrzeugaufb...,NaN,NaN,NaN,[],[],NaN,...,0,operational,"{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'items': [], 'total': 0}",deeptech,1.0,[],NaN,NaN
632,1660542,N√©olithe,n_olithe,Néolithe – Industrialiser l'écologie,NaN,2-10,4.0,[],[],seed,...,0,operational,"{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'items': [], 'total': 0}",deeptech,1.0,[],NaN,NaN
650,1482033,Akwatyx,akwatyx,Innovative company dedicated to aqua-fun water...,Water,NaN,NaN,[],[],NaN,...,0,operational,"{'items': [], 'total': 0}","{'items': [{'id': 1239811, 'name': 'IMT Starte...","{'items': [], 'total': 0}",non_deeptech,0.0,[],NaN,NaN
684,1835167,Eyye,eyye,NaN,NaN,NaN,NaN,[],[],NaN,...,0,operational,"{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'items': [], 'total': 0}",non_deeptech,0.0,[],NaN,NaN


### Imputer based on the age of the company only as the missing values of the industry amount to 212

In [53]:
selected_data = selected_data[selected_data["age"].notna()]

In [54]:
selected_data.shape

(1421, 28)

In [55]:
selected_data = selected_data[selected_data.launch_year >= 2010]

In [56]:
selected_data.shape

(1298, 28)

In [57]:
imputer = selected_data[["employees_latest", "launch_year"]].groupby(by=["launch_year"]).median()
imputer

,employees_latest
launch_year,
2010.0,24.0
2011.0,17.0
2012.0,11.5
2013.0,14.0
2014.0,13.0
2015.0,13.5
2016.0,12.5
2017.0,11.0
2018.0,10.0


In [58]:
imputer = imputer.to_dict(orient="index")

In [59]:
imputer

{2010.0: {'employees_latest': 24.0},
 2011.0: {'employees_latest': 17.0},
 2012.0: {'employees_latest': 11.5},
 2013.0: {'employees_latest': 14.0},
 2014.0: {'employees_latest': 13.0},
 2015.0: {'employees_latest': 13.5},
 2016.0: {'employees_latest': 12.5},
 2017.0: {'employees_latest': 11.0},
 2018.0: {'employees_latest': 10.0},
 2019.0: {'employees_latest': 6.0},
 2020.0: {'employees_latest': 8.0}}

In [60]:
def fill_missing_employees(n_employees, launch_year, imputer_dict):
    if np.isnan(n_employees):
        return imputer_dict[launch_year]['employees_latest']
    else:
        return n_employees

In [61]:
selected_data['employees_latest'] = selected_data.apply(
    lambda row: fill_missing_employees(
        row['employees_latest'], 
        row['launch_year'], 
        imputer), 
    axis=1
)


In [62]:
selected_data.employees_latest.isna().sum()

0

### computing the funding amounts employees ratio

In [63]:
def funding_amounts_employees(data):
    '''
    function that performs the ratio between funding and employees
    '''

    funding = data['total_funding_source']
    employees = data['employees_latest']

    return funding/employees

In [64]:
selected_data["funding_employees_ratio"] = funding_amounts_employees(selected_data)

# model

### fixing X_train et X_test -> database including only companies created after 2010 + target = [0, 0.5, 1]

In [65]:
selected_data.columns

Index(['id', 'name', 'path', 'tagline', 'about', 'employees',
       'employees_latest', 'industries', 'technologies', 'growth_stage',
       'tags', 'launch_year', 'launch_month', 'has_strong_founder',
       'has_super_founder', 'total_funding', 'total_funding_source',
       'last_funding', 'last_funding_source', 'company_status', 'team',
       'investors', 'fundings', 'deep_or_not', 'target', 'industry',
       'industry_simplified', 'age', 'funding_employees_ratio'],
      dtype='object')

X = selected_data.drop(columns = ['target', "deep_or_not"])
y = selected_data['target']

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

X_train.to_csv("bpideep/rawdata/X_train_after2010.xlsx")
y_train.to_csv("bpideep/rawdata/y_train_after2010.xlsx")
X_test.to_csv("bpideep/rawdata/X_test_after2010.xlsx")
y_test.to_csv("bpideep/rawdata/y_test_after2010.xlsx")

### train_test_split

####  almost deeptech = 0

In [4]:
X_train = pd.read_csv("bpideep/rawdata/X_train_after2010.xlsx")
y_train = pd.read_csv("bpideep/rawdata/y_train_after2010.xlsx")
#X_test = pd.read_csv("bpideep/rawdata/X_test_after2010.xlsx")
#y_test = pd.read_csv("bpideep/rawdata/y_test_after2010.xlsx")

X_train.drop(columns = 'Unnamed: 0', inplace= True)
y_train.drop(columns = 'Unnamed: 0', inplace= True)

X_train = X_train[["funding_employees_ratio"]]
y_train = y_train.values.ravel()

In [21]:
y_train[y_train == 0.5] = 0

In [22]:
model = LogisticRegression(C = 1.52, penalty = 'l1', solver= 'liblinear')
cv_result = cross_validate(model, X_train, y_train, cv=5, scoring=["accuracy", "recall", "f1", "precision"])

In [24]:
scoring=["test_accuracy", "test_recall", "test_f1", "test_precision"]

for metric in scoring:
    print(f"{metric} : {cv_result[metric].mean() * 100 : .2f}%")

test_accuracy :  62.52%
test_recall :  28.62%
test_f1 :  40.20%
test_precision :  68.47%


#### almost_deeptech == 2

In [25]:
X_train2 = pd.read_csv("bpideep/rawdata/X_train_after2010.xlsx")
y_train2 = pd.read_csv("bpideep/rawdata/y_train_after2010.xlsx")
X_train2.drop(columns = 'Unnamed: 0', inplace= True)
y_train2.drop(columns = 'Unnamed: 0', inplace= True)
X_train2 = X_train2[["funding_employees_ratio"]]
y_train2 = y_train2.values.ravel()

In [26]:
y_train2[y_train2 == 0.5] = 2

In [27]:
model = LogisticRegression(C = 1.52, penalty = 'l1', solver= 'liblinear')
cv_result = cross_validate(model, X_train, y_train, cv=5, scoring=["accuracy", "recall", "f1", "precision"])

In [28]:
scoring=["test_accuracy", "test_recall", "test_f1", "test_precision"]

for metric in scoring:
    print(f"{metric} : {cv_result[metric].mean() * 100 : .2f}%")

test_accuracy :  62.71%
test_recall :  29.06%
test_f1 :  40.71%
test_precision :  68.81%


# finetuning the feature funding_employees_ratio with the growth stage

In [69]:
selected_data[selected_data.growth_stage.isna()]

,id,name,path,tagline,about,employees,employees_latest,industries,technologies,growth_stage,...,company_status,team,investors,fundings,deep_or_not,target,industry,industry_simplified,age,funding_employees_ratio
1002,1482023,Argos VR Systems,argos_vr_systems,Rig system mixing DSLR cameras and Raspberry P...,NaN,NaN,11.0,[],[],NaN,...,operational,"{'items': [{'id': 1481010, 'name': 'Alexandre ...","{'items': [], 'total': 0}","{'items': [], 'total': 0}",non_deeptech,0.0,[],NaN,3.0,0.0
1244,1478711,Rollkers,rollkers,Rollkers is The world’s first Electronic “UNDE...,"In just three years since its foundation, the ...",NaN,11.5,"[{'id': 1275, 'name': 'sports'}, {'id': 100111...",[],NaN,...,operational,"{'items': [], 'total': 0}","{'items': [{'id': 882647, 'name': 'Bourgogne A...","{'items': [], 'total': 0}",non_deeptech,0.0,"['sports', 'transportation']",transportation,8.0,0.0
1458,1483772,Tidalys,tidalys,High performance Tidal turbines with a low cos...,Tidalys has developed a novel range of tidal t...,NaN,11.5,[],[],NaN,...,operational,"{'items': [], 'total': 0}","{'items': [], 'total': 0}","{'items': [], 'total': 0}",almost_deeptech,0.5,[],NaN,8.0,0.0


Drop the 3 companies that don't have a growth stage

In [70]:
selected_data.shape

(1298, 29)

In [72]:
selected_data = selected_data[selected_data.growth_stage.notna()]

In [83]:
imputer2 = selected_data[["employees_latest", "launch_year", "growth_stage"]].groupby(by=["launch_year", "growth_stage"]).median()
imputer2

employees_latest
launch_year growth_stage                  
2010.0      early growth              20.0
            late growth               79.0
            seed                       8.0
2011.0      early growth              20.0
            late growth               82.0
            seed                       6.0
2012.0      early growth              18.0
            late growth               80.0
            mature                     6.0
            seed                       5.0
2013.0      early growth              18.0
            late growth               76.0
            mature                     3.0
            seed                       6.0
2014.0      early growth              19.0
            late growth               67.0
            mature                     6.0
            seed                       5.0
2015.0      early growth              18.0
            late growth              103.5
            mature                    11.0
            seed                       5.0
2016.0      early growth              20.0
            late growth               97.0
            mature                    32.0
            seed                       5.0
2017.0      early growth              15.0
            late growth               57.0
            mature                    14.0
            seed                       7.0
2018.0      early growth              16.5
            late growth              179.0
            seed                       6.0
2019.0      early growth              12.0
            late growth               71.0
            seed                       6.0
2020.0      mature                     8.0
            seed                       6.0

In [84]:
imputer2 = imputer2.to_dict(orient="index")
imputer2

{(2010.0, 'early growth'): {'employees_latest': 20.0},
 (2010.0, 'late growth'): {'employees_latest': 79.0},
 (2010.0, 'seed'): {'employees_latest': 8.0},
 (2011.0, 'early growth'): {'employees_latest': 20.0},
 (2011.0, 'late growth'): {'employees_latest': 82.0},
 (2011.0, 'seed'): {'employees_latest': 6.0},
 (2012.0, 'early growth'): {'employees_latest': 18.0},
 (2012.0, 'late growth'): {'employees_latest': 80.0},
 (2012.0, 'mature'): {'employees_latest': 6.0},
 (2012.0, 'seed'): {'employees_latest': 5.0},
 (2013.0, 'early growth'): {'employees_latest': 18.0},
 (2013.0, 'late growth'): {'employees_latest': 76.0},
 (2013.0, 'mature'): {'employees_latest': 3.0},
 (2013.0, 'seed'): {'employees_latest': 6.0},
 (2014.0, 'early growth'): {'employees_latest': 19.0},
 (2014.0, 'late growth'): {'employees_latest': 67.0},
 (2014.0, 'mature'): {'employees_latest': 6.0},
 (2014.0, 'seed'): {'employees_latest': 5.0},
 (2015.0, 'early growth'): {'employees_latest': 18.0},
 (2015.0, 'late growth'): 

In [85]:
imputer2.keys()

dict_keys([(2010.0, 'early growth'), (2010.0, 'late growth'), (2010.0, 'seed'), (2011.0, 'early growth'), (2011.0, 'late growth'), (2011.0, 'seed'), (2012.0, 'early growth'), (2012.0, 'late growth'), (2012.0, 'mature'), (2012.0, 'seed'), (2013.0, 'early growth'), (2013.0, 'late growth'), (2013.0, 'mature'), (2013.0, 'seed'), (2014.0, 'early growth'), (2014.0, 'late growth'), (2014.0, 'mature'), (2014.0, 'seed'), (2015.0, 'early growth'), (2015.0, 'late growth'), (2015.0, 'mature'), (2015.0, 'seed'), (2016.0, 'early growth'), (2016.0, 'late growth'), (2016.0, 'mature'), (2016.0, 'seed'), (2017.0, 'early growth'), (2017.0, 'late growth'), (2017.0, 'mature'), (2017.0, 'seed'), (2018.0, 'early growth'), (2018.0, 'late growth'), (2018.0, 'seed'), (2019.0, 'early growth'), (2019.0, 'late growth'), (2019.0, 'seed'), (2020.0, 'mature'), (2020.0, 'seed')])

In [88]:
def fill_missing_employees2(n_employees, launch_year, growth, imputer_dict):
    if np.isnan(n_employees):
        return imputer_dict[(launch_year, growth)]['employees_latest']
    else:
        return n_employees

In [91]:
selected_data['employees_latest'] = selected_data.apply(
    lambda row: fill_missing_employees(
        row['employees_latest'], 
        row['launch_year'], 
        row['growth_stage'],
        imputer2), 
    axis=1
)


In [92]:
selected_data["funding_employees_ratio"] = funding_amounts_employees(selected_data)

## model

In [110]:
data[data.employees_latest.isna() & data.total_funding_source > 0]["id"]

400    1336864
579    1773238
628     232939
Name: id, dtype: int64

In [113]:
tmp = pd.read_csv("bpideep/rawdata/X_train_after2010.xlsx")
tmp[tmp.id == 1336864][["id","funding_employees_ratio","total_funding_source"]]

,id,funding_employees_ratio,total_funding_source
794,1336864,1346.740741,18181


In [121]:
X_train = pd.read_csv("bpideep/rawdata/X_train_after2010.xlsx")
y_train = pd.read_csv("bpideep/rawdata/y_train_after2010.xlsx")

In [123]:
tmp = pd.merge(X_train, y_train, left_index = True, right_index= True)

In [132]:
tmp = tmp[tmp.growth_stage.notna()]
tmp.shape

(1035, 30)

In [133]:
X_train = tmp[["id"]]
print(pd.merge(X_train, selected_data[["id", "funding_employees_ratio"]], left_on = "id", right_on= "id", how= "left"))
y_train = tmp["target"]

           id  funding_employees_ratio
0      898594            470087.000000
1      907487            134020.618557
2     1533567             83333.333333
3      156180            225000.000000
4     1836822                 0.000000
...       ...                      ...
1030  1662560                 0.000000
1031  1534628            490733.600000
1032   883391             83333.333333
1033   969347             20000.000000
1034  1483065             53571.428571

[1035 rows x 2 columns]


In [146]:
y_train0 = y_train
y_train0[y_train0 == 0.5] = 0

<ipython-input-146-13a2ed4a89c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train0[y_train0 == 0.5] = 0


In [147]:
model = LogisticRegression(C = 1.52, penalty = 'l1', solver= 'liblinear')
cv_result = cross_validate(model, X_train, y_train0, cv=5, scoring=["accuracy", "recall", "f1", "precision"])

/Users/catherinechen/.pyenv/versions/3.8.6/envs/deep4deep2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/catherinechen/.pyenv/versions/3.8.6/envs/deep4deep2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/catherinechen/.pyenv/versions/3.8.6/envs/deep4deep2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [148]:
scoring=["test_accuracy", "test_recall", "test_f1", "test_precision"]

for metric in scoring:
    print(f"{metric} : {cv_result[metric].mean() * 100 : .2f}%")

test_accuracy :  55.46%
test_recall :  0.00%
test_f1 :  0.00%
test_precision :  0.00%


In [158]:
data[data.launch_year >= 2010]["growth_stage"].isna().sum()

3

In [159]:
data[data.launch_year >= 2010].shape

(1298, 94)